इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड आधार](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%AB) में है %A04 -%E4%BD%BF%E7%94%A8ट्रांसफॉर्मर%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1)।

यदि आप कोलाब पर इस ज्यूपिटर नोटबुक को खोलते हैं, तो आपको 🤗ट्रांसफॉर्मर और 🤗डेटासेट इंस्टॉल करना होगा। विशिष्ट आदेश इस प्रकार है (यदि गति धीमी है तो टिप्पणी हटाएं और चलाएं, कृपया घरेलू स्रोत पर स्विच करें, और दूसरी पंक्ति में पैरामीटर जोड़ें)।

सेल चलाने से पहले, यह अनुशंसा की जाती है कि आप सीखने के लिए एक विशेष पायथन वातावरण स्थापित करने के लिए इस प्रोजेक्ट के रीडमी में दिए गए निर्देशों का पालन करें।

In [ ]:
#! पिप डेटासेट ट्रांसफार्मर स्थापित करें
# -i https://pypi.tona.tsinghua.edu.cn/simple

यदि आप इस ज्यूपिटर नोटबुक को अपनी स्थानीय मशीन पर खोल रहे हैं, तो कृपया सुनिश्चित करें कि आपके वातावरण में उपरोक्त पुस्तकालयों के नवीनतम संस्करण स्थापित हैं।

आप इस ज्यूपिटर नोटबुक की विशिष्ट पायथन स्क्रिप्ट फ़ाइल यहां [यहां] (https://github.com/huggingface/transformers/blob/master/examples/pytorch/multiple-choice/) पा सकते हैं, और आप वितरित का भी उपयोग कर सकते हैं विधि अपने मॉडल को बेहतर बनाने के लिए एकाधिक जीपीयू या टीपीयू का उपयोग करें।

# मॉडल को बेहतर बनाकर बहुविकल्पीय कार्य बनाएं

वर्तमान ज्यूपिटर नोटबुक में, हम बताएंगे कि एक बहु-विकल्प कार्य बनाने के लिए मनमाने ढंग से [🤗ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) मॉडल को कैसे ठीक किया जाए जो दिए गए कई उत्तरों में से सबसे अच्छे उत्तर का चयन करता है। एक उचित. हमने जो डेटासेट उपयोग किया है वह [SWAG](https://www.aclweb.org/anthology/D18-1009/) है, लेकिन आप प्रीप्रोसेसिंग प्रक्रिया को अन्य मल्टीपल-सेलेक्ट डेटासेट या अपने स्वयं के डेटा पर भी लागू कर सकते हैं। SWAG सामान्य ज्ञान तर्क के बारे में एक डेटा सेट है। प्रत्येक नमूना एक स्थिति का वर्णन करता है और फिर चार संभावित विकल्प देता है।

इस ज्यूपिटर नोटबुक को [मॉडल हब](https://huggingface.co/models) में किसी भी मॉडल पर चलाया जा सकता है, जब तक कि मॉडल में मल्टी-सेलेक्ट हेड संस्करण हो। आपके मॉडल और आपके द्वारा उपयोग किए जाने वाले GPU के आधार पर, आपको मेमोरी त्रुटियों से बचने के लिए बैच आकार को समायोजित करने की आवश्यकता हो सकती है। इन दो मापदंडों को सेट करने के बाद, बाकी ज्यूपिटर नोटबुक सुचारू रूप से चल सकती है:

In [2]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

## डेटा सेट लोड करें

हम डेटा डाउनलोड करने के लिए [🤗Datasets](https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे। इस प्रक्रिया को 'load_dataset' फ़ंक्शन का उपयोग करके आसानी से पूरा किया जा सकता है।

In [3]:
from datasets import load_dataset, load_metric

अगली बार चलाने पर इसे दोबारा डाउनलोड करने से बचने के लिए `load_dataset` डेटासेट को कैश कर देगा।

In [ ]:
datasets = load_dataset("swag", "regular")

Reusing dataset swag (/home/sgugger/.cache/huggingface/datasets/swag/regular/0.0.0/f9784740e0964a3c799d68cec0d992cc267d3fe94f3e048175eca69d739b980d)


इसके अलावा, आप इसे [लिंक](https://gas.graviti.cn/dataset/datawhale/SWAG) से भी डाउनलोड कर सकते हैं
) डेटा डाउनलोड करें और इसे डीकंप्रेस करें, तीन डीकंप्रेस्ड सीएसवी फ़ाइलों को `docs/Chapter 4-Using Transformers to Solve NLP Tasks/datasets/swag` निर्देशिका में कॉपी करें, और फिर उन्हें निम्नलिखित कोड के साथ लोड करें।

In [6]:
import os

data_path = './datasets/swag/'
cache_dir = os.path.join(data_path, 'cache')
data_files = {'train': os.path.join(data_path, 'train.csv'), 'val': os.path.join(data_path, 'val.csv'), 'test': os.path.join(data_path, 'test.csv')}
datasets = load_dataset(data_path, 'regular', data_files=data_files, cache_dir=cache_dir)

Using custom data configuration regular-2ab2d66f12115abf


Dataset swag downloaded and prepared to ./datasets/swag/cache/swag/regular-2ab2d66f12115abf/0.0.0/a16ae67faa24f4cdd6d1fc6bfc09bdb6dc15771716221ff8bacbc6cc75533614. Subsequent calls will reuse this data.


`डेटासेट` ऑब्जेक्ट स्वयं एक [`डेटासेटडिक्ट`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) है, जिसमें कुंजी-मूल्य जोड़े शामिल हैं (`mnli` एक विशेष उदाहरण है जिसमें बेमेल सत्यापन और परीक्षण सेट के लिए कुंजी-मूल्य जोड़े)।

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

किसी वास्तविक तत्व तक पहुँचने के लिए, आपको पहले एक विभाजन का चयन करना होगा, फिर एक सूचकांक देना होगा:

In [ ]:
datasets["train"][0]

{'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'fold-ind': '3416',
 'gold-source': 'gold',
 'label': 0,
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'video-id': 'anetv_jkn6uvmqwh4'}

डेटा कैसा दिखता है इसका अंदाज़ा लगाने के लिए, नीचे दिया गया फ़ंक्शन डेटासेट से कुछ यादृच्छिक रूप से चयनित उदाहरण प्रदर्शित करेगा।

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,ending0,ending1,ending2,ending3,fold-ind,gold-source,label,sent1,sent2,startphrase,video-id
0,are seated on a field.,are skiing down the slope.,are in a lift.,are pouring out in a man.,16668,gold,1,A man is wiping the skiboard.,Group of people,A man is wiping the skiboard. Group of people,anetv_JmL6BiuXr_g
1,performs stunts inside a gym.,shows several shopping in the water.,continues his skateboard while talking.,is putting a black bike close.,11424,gold,0,The credits of the video are shown.,A lady,The credits of the video are shown. A lady,anetv_dWyE0o2NetQ
2,is emerging into the hospital.,are strewn under water at some wreckage.,tosses the wand together and saunters into the marketplace.,swats him upside down.,15023,gen,1,"Through his binoculars, someone watches a handful of surfers being rolled up into the wave.",Someone,"Through his binoculars, someone watches a handful of surfers being rolled up into the wave. Someone",lsmdc3016_CHASING_MAVERICKS-6791
3,spies someone sitting below.,opens the fridge and checks out the photo.,puts a little sheepishly.,staggers up to him.,5475,gold,3,"He tips it upside down, and its little umbrella falls to the floor.","Back inside, someone","He tips it upside down, and its little umbrella falls to the floor. Back inside, someone",lsmdc1008_Spider-Man2-75503
4,carries her to the grave.,laughs as someone styles her hair.,sets down his glass.,stares after her then trudges back up into the street.,6904,gen,1,Someone kisses her smiling daughter on the cheek and beams back at the camera.,Someone,Someone kisses her smiling daughter on the cheek and beams back at the camera. Someone,lsmdc1028_No_Reservations-83242
5,stops someone and sweeps all the way back from the lower deck to join them.,is being dragged towards the monstrous animation.,"beats out many events at the touch of the sword, crawling it.",reaches into a pocket and yanks open the door.,14089,gen,1,"But before he can use his wand, he accidentally rams it up the troll's nostril.",The angry troll,"But before he can use his wand, he accidentally rams it up the troll's nostril. The angry troll",lsmdc1053_Harry_Potter_and_the_philosophers_stone-95867
6,sees someone's name in the photo.,gives a surprised look.,kneels down and touches his ripped specs.,spies on someone's clock.,8407,gen,1,Someone keeps his tired eyes on the road.,"Glancing over, he","Someone keeps his tired eyes on the road. Glancing over, he",lsmdc1024_Identity_Thief-82693
7,stops as someone speaks into the camera.,notices how blue his eyes are.,is flung out of the door and knocks the boy over.,"flies through the air, its a fireball.",4523,gold,1,Both people are knocked back a few steps from the force of the collision.,She,Both people are knocked back a few steps from the force of the collision. She,lsmdc0043_Thelma_and_Luise-68271
8,sits close to the river.,have pet's supplies and pets.,"pops parked outside the dirt facility, sending up a car highway to catch control.",displays all kinds of power tools and website.,8112,gold,1,A guy waits in the waiting room with his pet.,A pet store and its van,A guy waits in the waiting room with his pet. A pet store and its van,anetv_9VWoQpg9wqE
9,"the slender someone, someone turns on the light.",", someone gives them to her boss then dumps some alcohol into dough.","liquids from a bowl, she slams them drunk.",wags his tail as someone returns to the hotel room.,10867,gold,3,"Inside a convenience store, she opens a freezer case.",Dolce,"Inside a convenience store, she opens a freezer case. Dolce",lsmdc3090_YOUNG_ADULT-43871


डेटासेट में प्रत्येक उदाहरण में एक संदर्भ होता है, जिसमें पहला वाक्य (फ़ील्ड `sent1`) और दूसरे वाक्य का परिचय (फ़ील्ड `sent2`) शामिल होता है। फिर चार संभावित अंत दिए गए हैं (फ़ील्ड `एंडिंग 0`, `एंडिंग 1`, `एंडिंग 2` और `एंडिंग 3`) और मॉडल को सही एक चुनने के लिए कहा जाता है (फ़ील्ड `लेबल` द्वारा दर्शाया गया है)। निम्नलिखित फ़ंक्शन हमें एक उदाहरण को अधिक सहजता से देखने की अनुमति देता है:

In [ ]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [ ]:
show_one(datasets["train"][0])

Context: Members of the procession walk down the street holding small horn brass instruments.
  A - A drum line passes by walking down the street playing their instruments.
  B - A drum line has heard approaching them.
  C - A drum line arrives and they're outside dancing and asleep.
  D - A drum line turns the lead singer watches the performance.

Ground truth: option A


In [ ]:
show_one(datasets["train"][15])

Context: Now it's someone's turn to rain blades on his opponent.
  A - Someone pats his shoulder and spins wildly.
  B - Someone lunges forward through the window.
  C - Someone falls to the ground.
  D - Someone rolls up his fast run from the water and tosses in the sky.

Ground truth: option C


## डेटा प्रीप्रोसेसिंग

इन टेक्स्ट को मॉडल में इनपुट करने से पहले, हमें उन्हें प्रीप्रोसेस करना होगा। यह 🤗ट्रांसफॉर्मर के 'टोकनाइज़र' द्वारा किया जाता है, जो, जैसा कि इसके नाम से पता चलता है, इनपुट को टोकन के अनुक्रम के रूप में दर्शाता है और फिर पूर्व-प्रशिक्षित शब्दावली को देखकर उन्हें संबंधित आईडी में परिवर्तित करता है। अंत में, इसे मॉडल द्वारा अपेक्षित प्रारूप में परिवर्तित किया जाता है और मॉडल द्वारा आवश्यक अन्य इनपुट उत्पन्न किए जाते हैं।

यह सब करने के लिए, हम `AutoTokenizer` की `from_pretrained` विधि का उपयोग करके अपने टोकननाइज़र को तुरंत चालू करते हैं, जो यह सुनिश्चित करेगा:

- हमें उस मॉडल आर्किटेक्चर के अनुरूप एक टोकननाइज़र मिलता है जिसका हम उपयोग करना चाहते हैं,
-हमने इस विशिष्ट मॉडल को पूर्व-प्रशिक्षित करने के लिए उपयोग की जाने वाली शब्दावली डाउनलोड की।

साथ ही, शब्दावली को कैश कर दिया जाएगा ताकि अगली बार चलाने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

हम टोकननाइज़र लाइब्रेरी (जो रस्ट द्वारा समर्थित है) से तेज़ टोकननाइज़र का उपयोग करने के लिए एक पैरामीटर के रूप में `use_fast=True` पास करते हैं। ये तेज़ टोकनाइज़र लगभग सभी मॉडलों के साथ काम करते हैं, लेकिन अगर आपको पिछली कॉल में कोई त्रुटि मिलती है, तो उस पैरामीटर को हटा दें।

आप इस टोकननाइज़र को सीधे किसी वाक्य या वाक्य युग्म पर कॉल कर सकते हैं:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

आपके द्वारा चुने गए मॉडल के आधार पर, आपको उपरोक्त सेल में दिए गए शब्दकोश में अलग-अलग कुंजी-मूल्य जोड़े दिखाई देंगे। हम यहां जो कर रहे हैं उसके लिए वे महत्वपूर्ण नहीं हैं, बस यह जान लें कि वे उस मॉडल के लिए आवश्यक हैं जिसे हम बाद में इंस्टेंटिअट करेंगे। यदि आप रुचि रखते हैं, तो आप [इस ट्यूटोरियल](https://huggingface.co/transformers/preprocessing.html) में उनके बारे में अधिक जान सकते हैं।

जैसा कि नीचे दिए गए शब्दकोश में दिखाया गया है, डेटासेट को प्रीप्रोसेस करने के लिए, हमें वाक्यों वाले कॉलम के नाम जानने की जरूरत है:

हम अपने नमूनों को प्रीप्रोसेस करने के लिए एक फ़ंक्शन लिख सकते हैं। सबसे मुश्किल हिस्सा टोकननाइज़र को कॉल करने से पहले सभी संभावित वाक्य जोड़े को दो बड़ी सूचियों में डालना है, और फिर परिणामों को समतल करना है ताकि प्रत्येक उदाहरण में चार इनपुट आईडी, ध्यान मास्क इत्यादि हों।

`टोकनाइज़र` को कॉल करते समय, हम पैरामीटर `truncation=True` पास करते हैं। इससे यह सुनिश्चित हो जाएगा कि जो इनपुट चयनित मॉडल की क्षमता से अधिक लंबे हैं, उन्हें उस अधिकतम लंबाई तक छोटा कर दिया जाएगा जिसे मॉडल स्वीकार कर सकता है।

In [ ]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]

def preprocess_function(examples):
# दूसरे वाक्य की चार संभावनाओं के साथ चलने के लिए प्रत्येक पहले वाक्य को चार बार दोहराएं।
    first_sentences = [[context] * 4 for context in examples["sent1"]]
# प्रत्येक संदर्भ के लिए सभी संभव दूसरे वाक्य पकड़ें।
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
#सपाटसब कुछ
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
#टोकनीकरण
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
#अन-चपटा
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

यह फ़ंक्शन एक या कई उदाहरणों के साथ काम करता है, कई उदाहरणों के मामले में, टोकननाइज़र प्रत्येक कुंजी के लिए सूचियों की एक सूची लौटाएगा: सभी उदाहरणों की एक सूची (यहां 5), फिर सभी विकल्पों की एक सूची (4) और इनपुट आईडी की एक सूची (यहां लंबाई भिन्न हो सकती है क्योंकि हमने कोई पैडिंग लागू नहीं की है):

यह फ़ंक्शन एक या अधिक उदाहरणों का उपयोग कर सकता है. जब एकाधिक उदाहरण पारित किए जाते हैं, तो टोकननाइज़र प्रत्येक कुंजी के लिए सूचियों की एक सूची लौटाएगा: सभी उदाहरणों की एक सूची (लंबाई 5), फिर सभी विकल्पों की एक सूची (लंबाई 4) और इनपुट आईडी की एक सूची (लंबाई भिन्न होती है, क्योंकि हमने कोई पैडिंग लागू नहीं की):

In [ ]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [30, 25, 30, 28]


आइए दिए गए उदाहरण के लिए इनपुट को डीकोड करें:

In [ ]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession are playing ping pong and celebrating one left each in quick. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession wait slowly towards the cadets. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession play and go back and forth hitting the drums while the audience claps for them. [SEP]']

हम इसकी तुलना पहले से उत्पन्न जमीनी सच्चाई से कर सकते हैं:

In [ ]:
show_one(datasets["train"][3])

Context: A drum line passes by walking down the street playing their instruments.
  A - Members of the procession are playing ping pong and celebrating one left each in quick.
  B - Members of the procession wait slowly towards the cadets.
  C - Members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions.
  D - Members of the procession play and go back and forth hitting the drums while the audience claps for them.

Ground truth: option D


ये ठीक लगता है. हम पहले बनाए गए `डेटासेट` ऑब्जेक्ट की `मैप` विधि का उपयोग करके इस फ़ंक्शन को अपने डेटासेट के सभी उदाहरणों पर लागू कर सकते हैं। इसे 'डेटासेट' ऑब्जेक्ट के सभी शार्ड के सभी तत्वों पर लागू किया जाएगा, इसलिए हमारा प्रशिक्षण, सत्यापन और परीक्षण डेटा उसी तरह से प्रीप्रोसेस किया जाएगा।

In [ ]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/swag/regular/0.0.0/f9784740e0964a3c799d68cec0d992cc267d3fe94f3e048175eca69d739b980d/cache-975c81cf12e5b7ac.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/swag/regular/0.0.0/f9784740e0964a3c799d68cec0d992cc267d3fe94f3e048175eca69d739b980d/cache-d4806d63f1eaf5cd.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/swag/regular/0.0.0/f9784740e0964a3c799d68cec0d992cc267d3fe94f3e048175eca69d739b980d/cache-258c9cd71b0182db.arrow


इससे भी बेहतर, अगली बार जब आप इसे चलाते हैं तो इस चरण पर समय बर्बाद करने से बचने के लिए परिणाम 🤗डेटासेट लाइब्रेरी द्वारा स्वचालित रूप से कैश किए जाते हैं। 🤗डेटासेट लाइब्रेरी आमतौर पर इतनी स्मार्ट होती है कि यह पता लगा सकती है कि `मैप` में पास किया गया फ़ंक्शन कब बदल गया है (किस बिंदु पर कैश्ड डेटा का अब उपयोग नहीं किया जाता है)। उदाहरण के लिए, यह पता लगाएगा कि क्या आपने पहले सेल में कार्य बदला है और नोटबुक को फिर से चलाएँ। जब 🤗डेटासेट कैश फ़ाइलों का उपयोग करता है, तो यह संबंधित चेतावनियों का संकेत देता है। आप कैश फ़ाइलों का उपयोग न करने और प्रीप्रोसेसिंग को बाध्य करने के लिए `मैप` को कॉल करने में `load_from_cache_file=False` पास कर सकते हैं।

ध्यान दें कि हमने टेक्स्ट को बैच एनकोड करने के लिए `batched=True` पास किया है। यह हमारे द्वारा पहले लोड किए गए तेज़ टोकननाइज़र का लाभ उठाने के लिए है, जो पाठ को बैचों में समवर्ती रूप से संसाधित करने के लिए कई थ्रेड्स का उपयोग करेगा।

## मॉडल को फाइन-ट्यून करें

अब जब हमारा डेटा तैयार है, तो हम पूर्व-प्रशिक्षित मॉडल डाउनलोड कर सकते हैं और इसे फाइन-ट्यून कर सकते हैं। चूँकि हमारा कार्य एकाधिक विकल्पों के बारे में है, हम `AutoModelForMultipleChoice` वर्ग का उपयोग करते हैं। टोकननाइज़र की तरह, `from_pretrained` विधि हमारे लिए मॉडल को डाउनलोड और कैश करेगी।

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

यह चेतावनी हमें बताती है कि हम कुछ वज़न (`vocab_transform` और `vocab_layer_norm` लेयर्स) को हटा रहे हैं और कुछ अन्य पैरामीटर्स (`pre_classifier` और `classifier` लेयर्स) को बेतरतीब ढंग से प्रारंभ कर रहे हैं। यह पूरी तरह से सामान्य है, क्योंकि हमने मॉडल को पूर्व-प्रशिक्षित करते समय नकाबपोश भाषा मॉडलिंग के लिए उपयोग किए गए हेडर को त्याग दिया और इसे एक नए बहु-चयन हेडर के साथ बदल दिया, और हमारे पास इसके पूर्व-प्रशिक्षित वजन नहीं हैं, इसलिए यह चेतावनी बताती है कि इस मॉडल की आवश्यकता है अनुमान लगाने के लिए इसका उपयोग करने से पहले इसे ठीक किया जाना चाहिए, और हम बिल्कुल यही करने जा रहे हैं।

एक `ट्रेनर` को इंस्टेंट करने के लिए, हमें तीन अन्य चीजों को परिभाषित करने की आवश्यकता है। सबसे महत्वपूर्ण है [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), जो एक ऐसा वर्ग है जिसमें प्रशिक्षण के लिए उपयोग की जाने वाली सभी विशेषताएं शामिल हैं। इसे एक फ़ोल्डर नाम में पास करने की आवश्यकता है जहां मॉडल की चौकियां सहेजी जाएंगी, जबकि अन्य सभी पैरामीटर वैकल्पिक हैं:

In [ ]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

यहां हम प्रत्येक युग के अंत में मूल्यांकन स्थापित करते हैं, सीखने की दर को समायोजित करते हैं, ज्यूपिटर नोटबुक के शीर्ष पर परिभाषित `बैच_आकार` का उपयोग करते हैं, और प्रशिक्षण के लिए उपयोग किए जाने वाले युगों की संख्या, साथ ही वजन घटाने को अनुकूलित करते हैं।

फिर, हमें अपने `ट्रेनर` को यह बताना होगा कि प्रीप्रोसेस्ड इनपुट डेटा से बैच डेटा कैसे बनाया जाए। हमने अभी तक कोई पैडिंग नहीं की है, क्योंकि हम प्रत्येक बैच को बैच के भीतर अधिकतम लंबाई तक पैडिंग करेंगे (संपूर्ण डेटासेट की अधिकतम लंबाई का उपयोग करने के बजाय)। ये होगा *डेटा कोलेटर* का काम. यह उदाहरणों की एक सूची स्वीकार करता है और उन्हें एक बैच में परिवर्तित करता है (हमारे मामले में, पैडिंग लागू करके)। चूंकि हमारी विशिष्ट समस्या को संभालने के लिए लाइब्रेरी में कोई डेटा कोलेटर नहीं है, इसलिए यहां हम 'DataCollatorWithPadding' के आधार पर एक को अनुकूलित करते हैं:

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
#अन-चपटा
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
# लेबल वापस जोड़ें
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

जब उदाहरणों की सूची में पारित किया जाता है, तो यह बड़ी सूची में सभी इनपुट/ध्यान मास्क आदि को समतल कर देता है और उन्हें `टोकनाइज़र.पैड` विधि में भेज देता है। यह एक बड़े टेंसर के साथ एक शब्दकोश लौटाता है (इसका आकार `(बैच_आकार * 4) x seq_length` है), जिसे हम फिर अनियंत्रित करते हैं।

हम जांच सकते हैं कि डेटा कोलेटर फीचर सूची पर ठीक से काम कर रहा है या नहीं, यहां हमें केवल उन सभी इनपुट सुविधाओं को हटाना सुनिश्चित करना है जो हमारे मॉडल द्वारा स्वीकार नहीं किए जाते हैं (यह वही है जो `ट्रेनर` हमारे लिए स्वचालित रूप से करता है):

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

फिर, यह सब चपटा और बिना चपटा संभावित त्रुटियों का स्रोत हो सकता है, तो आइए इनपुट पर एक और विवेक जांच करें:

In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

['[CLS] someone walks over to the radio. [SEP] someone hands her another phone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone takes the drink, then holds it. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone looks off then looks at someone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone stares blearily down at the floor. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [ ]:
show_one(datasets["train"][8])

Context: Someone walks over to the radio.
  A - Someone hands her another phone.
  B - Someone takes the drink, then holds it.
  C - Someone looks off then looks at someone.
  D - Someone stares blearily down at the floor.

Ground truth: option D


सब ठीक काम करता है!

अंत में, 'प्रशिक्षक' के लिए परिभाषित करें कि भविष्यवाणियों के आधार पर मूल्यांकन संकेतकों की गणना कैसे करें। हमें एक फ़ंक्शन को परिभाषित करने की आवश्यकता है जो हमारे द्वारा पहले लोड किए गए मीट्रिक का उपयोग करेगा, हमें जो एकमात्र प्रीप्रोसेसिंग करना है वह हमारे अनुमानित लॉग का argmax लेना है:

In [ ]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

फिर हमें बस यह सब अपने डेटासेट के साथ `ट्रेनर` में पास करना होगा:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

अब, हम 'ट्रेन' विधि को कॉल करके मॉडल को बेहतर बना सकते हैं:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.154598,0.828017,0.766520
2,0.296633,0.667454,0.786814
3,0.111786,0.994927,0.789363


TrainOutput(global_step=6897, training_loss=0.19714653808275168)

अंत में, अपने मॉडल को [🤗 मॉडल सेंटर](https://huggingface.co/models) पर [अपलोड](https://huggingface.co/transformers/model_sharing.html) करना न भूलें।